# Comparing Multi-Agent System to a Single-Model Baseline for Code Generation

This notebook presents a comparative study evaluating the performance of a multi-agent system for automated code generation against a single large language model (LLM) baseline. The multi-agent system, designed with specialized agents for tasks such as preprocessing, retrieval, planning, coding, and debugging, aims to tackle complex coding challenges more effectively than a monolithic single-model approach.

The objective of this study is to quantify the difference in performance between the two approaches across a set of coding problems. We will evaluate performance based on metrics such as the success rate in generating correct code, the time taken to produce a solution, and potentially other relevant factors like the number of iterations or debugging steps.

This notebook will cover:

1.  **Setup:** Loading necessary libraries, configuring the environment, and preparing the coding problems for evaluation.
2.  **Baseline (Single Model):** Implementing and running the single-model approach on the chosen coding problems.
3.  **Multi-Agent System:** Integrating and running the existing multi-agent system within the notebook environment.
4.  **Evaluation:** Running both approaches on the same set of problems and collecting performance data.
5.  **Analysis:** Comparing the collected data to draw conclusions about the effectiveness of each approach.
6.  **Results and Discussion:** Presenting the findings and discussing the implications of the results.


## Setup

In [29]:
import os
import sys
import dotenv
import json
from loguru import logger

# Append the models path in order to import the models
PROJECT_ROOT = os.path.join(os.getcwd(), 'src/')
print(PROJECT_ROOT)
sys.path.append(PROJECT_ROOT)

# Load env variables
dotenv.load_dotenv()

# Set log level INFO
logger.remove()
logger.add(sys.stderr, level="INFO")

/home/twanh/workspace/thesis/thesis-advent-of-agents/src/


2

In [30]:
# Import models from the system
from models.base_model import BaseLanguageModel
from models.gemini_model import GeminiLanguageModel
from models.openai_model import OpenAILanguageModel
from models.deepseek_model import DeepseekLanguageModel

### Load puzzles and input/outputs

In [31]:
# Get the correct paths
test_data_folder = os.path.join(PROJECT_ROOT, '..', 'experiments', 'test_data')
puzzles_folder = os.path.join(test_data_folder, 'puzzles/')
input_output_file = os.path.join(test_data_folder, 'answers2024.json')
puzzle_files = [os.path.join(puzzles_folder, f) for f in os.listdir(puzzles_folder) if os.path.isfile(os.path.join(puzzles_folder, f))]

In [32]:
# Create a datastructure were we can get by day
json_data = {}
with open(input_output_file, 'r') as f:
    json_data = {item['day']: item for item in json.load(f)}

puzzle_data = []
for file_path in puzzle_files:
    # Get the day of the puzzle file
    file_name = os.path.basename(file_path)
    day_str = file_name.split('_')[-1].split('.')[0]
    day = int(day_str)

    if day in json_data:
        with open(file_path, 'r') as f:
            puzzle_description = f.read()

        puzzle_info = {
            "year": json_data[day]['year'],
            "day": day,
            "description": puzzle_description,
            "input": json_data[day]['input'],
            "expected_output": json_data[day]['part1']
        }

        puzzle_data.append(puzzle_info)

# Sort by day
puzzle_data.sort(key=lambda x: x['day'])
print(len(puzzle_data)) # should be 25

25


### Model Configurations

Create the configurations for the baseline and the system to use. Since the advent of agents system can use multiple models for each agent, the model used for each agent will be the same as the single model used for the baseline.

The models that will be tested are:

<!-- TODO: Update model list -->
- Gemini
- OpenAI

In [ ]:
from main import _get_model

models_to_test = ('gemini-2.0-flash', 'gemini-2.5-pro-preview-05-06' ,'gpt-4.1-mini', 'o4-mini', 'o3-mini')

models_to_test = ("gpt-4.1-mini",)

configs = [_get_model(model) for model in models_to_test]
print(configs)

## Baseline

### Baseline prompt

The prompt is based on the prompt the advent of agents system uses. However the baseline has no acess to the information the other agents provide. So there is only the `full_description` that provides the model with the puzzle. 
The steps in the prompt are the same expect the first step is removed, which was to analyze the generated plan.

In [36]:
BASELINE_PROMPT = """
# Advent of Code Implementation Agent

You are an expert coding agent specializing in implementing solutions for Advent of Code puzzles.
Your task is to convert a detailed solution plan into clean, efficient, and correct Python code that solves the given problem.
You excel at translating algorithmic plans into precise implementations.


It will be provided as the following JSON

```json
{{
    "full_description": "The full description of the problem (string)",
}}
```

## YOUR RESPONSIBILITIES

Your primary goal is to produce a complete, correct, and efficient Python implementation that:

1. Correctly solves both the provided examples and will work for the actual puzzle input
2. Follows good software engineering practices
3. Includes appropriate comments and documentation
4. Handles edge cases and potential errors
5. Is executable via command line as: `python3 [program].py [puzzleinputfile]`

## IMPLEMENTATION PROCESS

Follow these steps meticulously:

-----------------------------------------
STEP 1. Design Your Code Structure
-----------------------------------------

- Create a clear, modular structure with well-named functions matching the plan's major steps
- Define appropriate data structures with explicit type hints
- Plan your function signatures and interfaces before implementation
- Use the keywords and underlying concepts to think about what algoritms to use to solve the problems.

-----------------------------------------
STEP 2. Implement Core Logic
-----------------------------------------

- Write robust implementations of all algorithms described in the plan
- Include detailed comments explaining complex logic
- Follow Python best practices (PEP 8, appropriate naming conventions)
- Use type hints throughout your code


-----------------------------------------
STEP 3. Handle Edge Cases Explicitly
-----------------------------------------

- Add specific code to handle all edge cases mentioned in the plan
- Anticipate and handle additional edge cases common in Advent of Code:
  - Empty input
  - Boundary conditions (min/max values)
- Use the test cases to reason about your code and make sure it would solve the test cases correctly

----------------------------------------
STEP 4. Test Against Examples
----------------------------------------

- Include code that runs and validates against all provided examples
- Add assertions to verify intermediate results match expected values
- Print debugging information that would help diagnose issues to STDERR
    - STDOUT can only be used to print the final result.

----------------------------------------
STEP 5. Optimize If Necessary
-----------------------------------------

- Review your solution for performance bottlenecks
- Apply optimizations where appropriate, explaining your choices
- Ensure the solution will scale to handle the full problem input

-----------------------------------------
STEP 6. Finalize Solution
-----------------------------------------

- Ensure your code has a clear entry point (typically a `main()` function)
- Include code to read from the puzzle input file specified as a command-line argument
- Make sure that your code follows the proper structure as documented (example code template) below.
- Add a brief summary comment at the top explaining the approach
- Verify all functions have appropriate docstrings


-----------------------------------------
OUTPUT FORMAT
-----------------------------------------
Your response must be a valid JSON object with the following structure:

The generated code should be provided as the value of the code key in the JSON object. Ensure that the code is properly escaped to be a valid JSON string. This means that any double quotes within the code should be escaped with a backslash (\"), and newlines should be represented as \\n

```json
{{
  "code": "Complete Python code as a string with all necessary formatting. MAKE SURE THAT THIS IS VALID JSON"
}}
```


## EXAMPLE CODE TEMPLATE

```python
\"\"\"
Advent of Code [Year] Day [Number]: [Title]
Solution implementation based on the provided plan.

Usage: python3 solution.py [input_file]
\"\"\"
from typing import List, Dict, Tuple, Set, Optional
import sys
from collections import defaultdict, deque
import re
# Import other necessary libraries

def parse_input(input_file: str) -> [appropriate_return_type]:
    "\"\"Parse the puzzle input from file into appropriate data structures.

    Args:
        input_file: Path to the input file

    Returns:
        [Description of return value]
    \"\"\"
    with open(input_file, 'r') as f:
        # Process file content
        pass
    # Implementation...

def solve_part_one(parsed_data: [type]) -> [type]:
    \"\"\"Solve part one of the puzzle.

    Args:
        parsed_data: Processed input data

    Returns:
        Solution for part one
    \"\"\"
    # Implementation...

def main():
    # Check command line arguments
    if len(sys.argv) < 2:
        print("Usage: python3 solution.py [input_file]")
        return

    input_file = sys.argv[1]

    # Parse input
    parsed_data = parse_input(input_file)

    # Solve part one
    part_one_solution = solve_part_one(parsed_data)
    # ONLY PRINT THE RESULT, NO OTHER TEXT
    print(part_one_solution)

    # Test with examples (if available)
    # [Example testing code]

if __name__ == "__main__":
    main()
```

Remember to follow the plan closely while filling in implementation details that the planner may have omitted. Your goal is to bridge the gap between algorithmic description and working code.


-----------------------------------

Your input is:

{json_input}

"""

### Run baseline

In [37]:
from utils.utils import extract_json_from_markdown
from agents.debugging_agent import DebuggingAgent
from utils.util_types import TestCase

def run_and_test_baseline(puzzle: str, puzzle_input: str, expected_output: str, model: BaseLanguageModel) -> bool:

    # Create the prompts
    json_inp = json.dumps({'full_description': puzzle})
    prompt = BASELINE_PROMPT.format(json_input=json_inp)
    # Prompt the model
    resp = model.prompt(prompt)

    # Extract the solution
    try:
        code = json.loads(extract_json_from_markdown(resp)[0]).get('code')
    except json.JSONDecodeError:
        print("Failed because decode")
        # TODO: Add retrying?
        return False, "NO CODE"

    # Use debugging agent to test the final solution
    dba = DebuggingAgent(
        'debugging',
        model=model,
        expected_output=expected_output,
        puzzle_input=puzzle_input,
    )
    run_result = dba._run_test(
        code,
        TestCase(
            input_=puzzle_input,
            expected_output=expected_output,
        ),
    )

    return run_result.success, code


In [ ]:
# baseline_results = {}
# total = 0
# solved = 0
# # TODO: remove slice when running full tests
# for config in configs[:1]:
#     baseline_model = config['baseline']
#     baseline_results[baseline_model.model_name] = {}
#     print(f"Using model: {baseline_model.model_name}")
#     for puzzle in puzzle_data:
#         total += 1
#         print(f"Running day {puzzle['day']}")
#         success = run_and_test_baseline(
#             puzzle=puzzle['description'],
#             puzzle_input=puzzle['input'],
#             expected_output=str(puzzle['expected_output']),
#             model=baseline_model
#         )
#         baseline_results[baseline_model.model_name][puzzle['day']] = success
#         if success:
#             print("Solved!")
#             solved += 1
#         else:
#             print("Not solved")

# print(f"Solved: {solved}/{total}")

Using model: gemini-2.0-flash
Running day 1


2025-05-19 11:28:45.169 | INFO     | agents.debugging_agent:_run_test:337 - Running code with test case
2025-05-19 11:28:45.172 | INFO     | agents.debugging_agent:_run_code:307 - Running code
2025-05-19 11:28:45.244 | INFO     | agents.debugging_agent:_run_test:345 - Test case is successful 1646452=1646452


Solved!
Running day 2


2025-05-19 11:28:49.242 | INFO     | agents.debugging_agent:_run_test:337 - Running code with test case
2025-05-19 11:28:49.245 | INFO     | agents.debugging_agent:_run_code:307 - Running code
2025-05-19 11:28:49.364 | INFO     | agents.debugging_agent:_run_test:345 - Test case is successful 524=524


Solved!
Running day 3
Failed because decode
Not solved
Running day 4


...

Not solved
Running day 24


2025-05-19 11:31:11.193 | INFO     | agents.debugging_agent:_run_test:337 - Running code with test case
2025-05-19 11:31:11.195 | INFO     | agents.debugging_agent:_run_code:307 - Running code
2025-05-19 11:31:11.264 | INFO     | agents.debugging_agent:_run_test:345 - Test case is successful 54715147844840=54715147844840


Solved!
Running day 25


2025-05-19 11:31:17.338 | INFO     | agents.debugging_agent:_run_test:337 - Running code with test case
2025-05-19 11:31:17.342 | INFO     | agents.debugging_agent:_run_code:307 - Running code
2025-05-19 11:31:17.433 | WARNING  | agents.debugging_agent:_run_test:357 - Test case was not successful
2025-05-19 11:31:17.433 | INFO     | agents.debugging_agent:_run_test:358 - Got: 22250, expected: 2900


Not solved
Solved: 11/25


## System

In [38]:
from agents.base_agent import BaseAgent
from agents.coding_agent import CodingAgent
from agents.debugging_agent import DebuggingAgent
from agents.planning_agent import PlanningAgent
from agents.pre_processing_agent import PreProcessingAgent
from agents.retreival_agent import RetrievalAgent
from core.orchestrator import Orchestrator
from utils.util_types import AgentSettings
from core.state import MainState
from utils.util_types import Puzzle

In [39]:
def setup_system(model: BaseLanguageModel, puzzle_input: str, expected_output:str) -> Orchestrator:
    agents = (
        (
            PreProcessingAgent(
                'preprocess', model=model,
            ),
            AgentSettings(enabled=True, can_debug=False),
        ),
        (
            RetrievalAgent(
                'retreival',
                model=model,
                connection_string=os.getenv('DB_CONNECTION_STRING') or '',
                openai_key=os.getenv('OPENAI_API_KEY') or '',
                # Use default weights
                weights=None,
            ),
            AgentSettings(enabled=True, can_debug=False),
        ),
        (
            PlanningAgent(
                'planning',
                model=model,
                n_plans=3,
            ),
            AgentSettings(enabled=True, can_debug=False),
        ),
        (
            CodingAgent('coding', model=model),
            AgentSettings(enabled=True, can_debug=False),
        ),
        (
            DebuggingAgent(
                'debugging',
                model=model,
                expected_output=expected_output,
                puzzle_input=puzzle_input,
            ),
            AgentSettings(enabled=True, can_debug=True),
        ),
    )

    orchestrator = Orchestrator(agents, {})
    return orchestrator


In [40]:
def run__and_test_system(day: int, puzzle_desc: str, puzzle_input: str, expected_output: str, model: BaseLanguageModel) -> tuple[bool, str]:

    orch = setup_system(model, puzzle_input, expected_output)

    puzzle = Puzzle(
        description=puzzle_desc,
        solution=None,
        year = 2024,
        day=day,

    )

    state = MainState(puzzle=puzzle)
    ret_state = orch.solve_puzzle(state)

    return ret_state.is_solved, ret_state.final_code

In [ ]:
# system_results = {}
# system_total = 0
# system_solved = 0
# # TODO: remove slice when running full tests
# for config in configs[:1]:
#     baseline_model = config['baseline']
#     system_results[baseline_model.model_name] = {}
#     print(f"Using model: {baseline_model.model_name}")
#     for puzzle in puzzle_data:
#         total += 1
#         print(f"Running day {puzzle['day']}")
#         success = run__and_test_system(
#             puzzle['day'], puzzle['description'], puzzle['input'], puzzle['expected_output'], baseline_model
#         )
#         system_results[baseline_model.model_name][puzzle['day']] = success
#         if success:
#             solved += 1

# print(f"Solved: {solved}/{total}")

2025-05-19 11:50:19.448 | INFO     | core.retreival:init_db:154 - Database initialization complete.
2025-05-19 11:50:19.454 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: preprocess


Using model: gemini-2.0-flash
Running day 1


2025-05-19 11:50:21.825 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: retreival
2025-05-19 11:50:45.588 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: planning
2025-05-19 11:50:45.593 | INFO     | agents.planning_agent:process:149 - Generating 3 plans
2025-05-19 11:50:45.595 | INFO     | agents.planning_agent:process:152 - Creating plan 1/3
2025-05-19 11:50:49.913 | WARNING  | agents.planning_agent:_generate_solution_plan:98 - Could not extract markdown plan from response.generated_plan=[] ret='## 1. Overall Approach\nThe problem asks to calculate the total distance between two lists of integers. The lists are paired by sorting both lists and then calculating the absolute difference between the elements at the same index in the sorted lists. The total distance is the sum of these absolute differences.\n\n## 2. Data Structures\n*   `list`: Used to store the input integers for both lists.\n*   `list`: Used to store sorted versions of both input lists.\n

Solved puzzle
Running day 2


2025-05-19 11:51:09.283 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: retreival
2025-05-19 11:51:23.365 | WARNING  | agents.retreival_agent:process:120 - Could not decode json: e=JSONDecodeError('Invalid control character at: line 7 column 722 (char 1266)')
2025-05-19 11:51:23.367 | WARNING  | agents.base_agent:_invalid_response_retry:66 - Retrying 1/3 for retreival
2025-05-19 11:51:42.538 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: planning
2025-05-19 11:51:42.544 | INFO     | agents.planning_agent:process:149 - Generating 3 plans
2025-05-19 11:51:42.548 | INFO     | agents.planning_agent:process:152 - Creating plan 1/3
2025-05-19 11:51:49.017 | WARNING  | agents.planning_agent:_generate_solution_plan:98 - Could not extract markdown plan from response.generated_plan=[] ret='```markdown\n## 1. Overall Approach\nThe problem requires analyzing reports, each a list of numbers, and determining how many are "safe". A report is safe if the numbers are ei

...

Solved puzzle
Running day 6


2025-05-19 11:55:29.490 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: retreival
2025-05-19 11:55:53.171 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: planning
2025-05-19 11:55:53.173 | INFO     | agents.planning_agent:process:149 - Generating 3 plans
2025-05-19 11:55:53.174 | INFO     | agents.planning_agent:process:152 - Creating plan 1/3
2025-05-19 11:56:02.306 | INFO     | agents.planning_agent:process:152 - Creating plan 2/3
2025-05-19 11:56:13.316 | INFO     | agents.planning_agent:process:152 - Creating plan 3/3
2025-05-19 11:56:21.800 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: coding
2025-05-19 11:56:28.167 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: debugging
2025-05-19 11:56:28.168 | INFO     | agents.debugging_agent:_run_test:337 - Running code with test case
2025-05-19 11:56:28.170 | INFO     | agents.debugging_agent:_run_code:307 - Running code
2025-05-19 11:56:28.232 | WARNING  | agents.debugging_

Solved puzzle
Running day 7


2025-05-19 11:57:38.494 | INFO     | core.retreival:init_db:154 - Database initialization complete.
2025-05-19 11:57:38.496 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: preprocess
2025-05-19 11:57:42.177 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: retreival
2025-05-19 11:57:50.020 | WARNING  | agents.retreival_agent:process:81 - Did not find any solutions for puzzle1-2017. Skipping...
2025-05-19 11:57:55.384 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: planning
2025-05-19 11:57:55.387 | INFO     | agents.planning_agent:process:149 - Generating 3 plans
2025-05-19 11:57:55.389 | INFO     | agents.planning_agent:process:152 - Creating plan 1/3


KeyboardInterrupt: 

## Comparing baseline vs system

In [41]:
import time
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [42]:
results = []

for model in configs:

    for puzzle in puzzle_data:
    
        puzzle_day = puzzle['day']
        puzzle_description = puzzle['description']
        input_ = puzzle['input']
        expected_ouptut = puzzle['expected_output']

        # Run single model on puzzle
        print(f"---- RUNNING SINGLE MODEL ON DAY {puzzle_day} WITH {model.model_name} ----")

        bl_start_time = time.time()
        bl_suc, bl_code = run_and_test_baseline(puzzle_description, input_, expected_ouptut, model)
        bl_end_time = time.time()

        print(f"Baseline Success: {bl_suc}")

        results.append({
            "day": puzzle_day,
            "approach": 'single-model',
            "model": model.model_name,
            "success": bl_suc,
            "time_taken": bl_end_time - bl_start_time,
            "code": bl_code
        })

        # Run system on puzzle
        print(f"---- RUNNING AOA SYSTEM ON DAY {puzzle_day} WITH {model.model_name} ----")
        sys_start_time = time.time()
        sys_suc, sys_code = run__and_test_system(puzzle_day, puzzle_description, input_, expected_ouptut, model)
        sys_end_time = time.time()

        results.append({
            "day": puzzle_day,
            "approach": 'system',
            "model": model.model_name,
            "success": sys_suc,
            "time_taken": sys_end_time - sys_start_time,
            "code": sys_code
        })

results_df = pd.DataFrame(results)

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
results_file_path = f'results-base-line-gpt-4.1-mini-{timestamp}'

results_df.to_csv(results_file_path, index=False)

---- RUNNING SINGLE MODEL ON DAY 1 WITH gpt-4.1-mini ----


2025-05-19 15:53:33.798 | INFO     | agents.debugging_agent:_run_test:337 - Running code with test case
2025-05-19 15:53:33.801 | INFO     | agents.debugging_agent:_run_code:307 - Running code
2025-05-19 15:53:33.864 | INFO     | agents.debugging_agent:_run_test:345 - Test case is successful 1646452=1646452
2025-05-19 15:53:34.033 | INFO     | core.retreival:init_db:154 - Database initialization complete.
2025-05-19 15:53:34.034 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: preprocess


Baseline Success: True
---- RUNNING AOA SYSTEM ON DAY 1 WITH gpt-4.1-mini ----


2025-05-19 15:53:39.501 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: retreival
2025-05-19 15:54:34.278 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: planning
2025-05-19 15:54:34.283 | INFO     | agents.planning_agent:process:149 - Generating 3 plans
2025-05-19 15:54:34.285 | INFO     | agents.planning_agent:process:152 - Creating plan 1/3
2025-05-19 15:54:51.269 | INFO     | agents.planning_agent:process:152 - Creating plan 2/3
2025-05-19 15:55:37.659 | INFO     | agents.planning_agent:process:152 - Creating plan 3/3
2025-05-19 15:55:53.113 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: coding
2025-05-19 15:56:02.862 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: debugging
2025-05-19 15:56:02.863 | INFO     | agents.debugging_agent:_run_test:337 - Running code with test case
2025-05-19 15:56:02.865 | INFO     | agents.debugging_agent:_run_code:307 - Running code
2025-05-19 15:56:02.934 | INFO     | agents.debugging_

---- RUNNING SINGLE MODEL ON DAY 2 WITH gpt-4.1-mini ----


...

---- RUNNING SINGLE MODEL ON DAY 4 WITH gpt-4.1-mini ----


2025-05-19 16:00:14.790 | INFO     | agents.debugging_agent:_run_test:337 - Running code with test case
2025-05-19 16:00:14.793 | INFO     | agents.debugging_agent:_run_code:307 - Running code
2025-05-19 16:00:14.844 | WARNING  | agents.debugging_agent:_run_test:357 - Test case was not successful
2025-05-19 16:00:14.845 | INFO     | agents.debugging_agent:_run_test:358 - Got: , expected: 2464
2025-05-19 16:00:14.937 | INFO     | core.retreival:init_db:154 - Database initialization complete.
2025-05-19 16:00:14.938 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: preprocess


Baseline Success: False
---- RUNNING AOA SYSTEM ON DAY 4 WITH gpt-4.1-mini ----


2025-05-19 16:00:21.112 | INFO     | core.orchestrator:solve_puzzle:58 - Running agent: retreival


IndexError: list index out of range